In [1]:
# Import necessary modules
import boto3
import sagemaker
import os

In [2]:
from sagemaker import get_execution_role
session =sagemaker.Session()
# store the current SageMaker session
# get IAM role
role=get_execution_role()
print(role)

bucket_name=session.default_bucket()
prefix = 'cancer-class'

arn:aws:iam::172268057478:role/service-role/AmazonSageMaker-ExecutionRole-20210122T150167


In [3]:
# set prefix, a descriptive name for a directory for our train test data
data_dir = '../Capstone Project/data/'
prefix = 'cancer-class'
# upload all data to S3
test_location = session.upload_data(os.path.join(data_dir,'test.csv'),key_prefix=prefix)
train_location= session.upload_data(os.path.join(data_dir,'train.csv'),key_prefix=prefix)

In [4]:
# Create an estimtor
# your import and estimator code, here

from sagemaker.pytorch import PyTorch
output_path = 's3://{}/{}'.format(bucket_name, prefix)
estimator = PyTorch(entry_point="train.py", 
                    source_dir="source_pytorch", 
                    role=role,
                    framework_version='1.0',
                    py_version = 'py3',
                    sagemaker_session = session,
                    output_path = output_path,
                    train_instance_count=1, 
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters= {'input_features':5,
                                      'hidden_dim':4, #change this to 6
                                      'output_dim':1,
                                      'epochs':160
                                     }
                   )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [5]:
%time

# Train your estimator on S3 training data
s3_input_train = sagemaker.TrainingInput(s3_data = train_location, content_type='csv')
estimator.fit({'train':s3_input_train})

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs
2021-03-15 00:03:49 Starting - Starting the training job...
2021-03-15 00:04:13 Starting - Launching requested ML instancesProfilerReport-1615766628: InProgress
......
2021-03-15 00:05:14 Starting - Preparing the instances for training.........
2021-03-15 00:06:49 Downloading - Downloading input data
2021-03-15 00:06:49 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-03-15 00:07:52,664 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-03-15 00:07:52,692 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-03-15 00:07:52,693 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-03-15 00:07:52,990 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-03-15 00

In [6]:
# deploy your model to create a predictor
%time
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.91 µs
---------------!

In [7]:
import os
import pandas as pd

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)
# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [8]:
from sklearn.metrics import accuracy_score
import numpy as np
test_y_preds = predictor.predict(test_x.values.astype(np.float32))
test_y_preds = [round(num) for num in test_y_preds.squeeze()]

In [9]:
# debug for overfitting
import numpy as np
train_data = pd.read_csv(os.path.join(data_dir, "train.csv"), header=None, names=None)
train_y = train_data.iloc[:,0]
train_x = train_data.iloc[:,1:]
train_y_preds = predictor.predict(train_x.values.astype(np.float32))
train_y_preds = [round(num) for num in train_y_preds.squeeze()]
accuracy = accuracy_score(train_y, train_y_preds)
print("%4.2f" % (100*accuracy), "%" )

96.98 %


In [10]:
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, test_y_preds)

print("%4.2f %%" % (100* accuracy) )


## print out the array of predicted and true labels, if you want
'''
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')

print(test_y.values)
'''

97.66 %


"\nprint('\nPredicted class labels: ')\nprint(test_y_preds)\nprint('\nTrue class labels: ')\n\nprint(test_y.values)\n"